# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = '../../ADA2017-Tutorials/02 - Intro to Pandas/Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.
import glob

We start by creating a frame per country containing all the data from the corresponding files.

In [ ]:
def folder_to_dataframe(folder):
    return pd.concat(map(pd.read_csv, glob.glob(DATA_FOLDER + folder + '/*.csv')))

frameSL, frameLI, frameGU = list(map(folder_to_dataframe, ('/ebola/sl_data/', '/ebola/liberia_data/', '/ebola/guinea_data/')))

In order to understand the data we have to work with, we display a summary for each data frame.

In [ ]:
frameSL.head()

In [ ]:
frameLI.head()

In [ ]:
frameGU.head()

We notice that the names of columns identifying dates/Dates, Description/Variable and National/Totals are not unified accross the three dataframes so we change that to have the same identifiers. We also notice that the date format in the dataset for Liberia does not correspond to the one used in the other datasets.

In [ ]:
frameSL.rename(columns={'date':'Date', 'variable':'Variable'}, inplace=True) #unifying column names
frameLI['Date'] = pd.to_datetime(frameLI.Date) #unifying the date format
frameSL['Date'] = pd.to_datetime(frameSL.Date) #unifying the date format
frameGU['Date'] = pd.to_datetime(frameGU.Date) #unifying the date format
frameGU.rename(columns={'Totals':'National','Description':'Variable' }, inplace=True) #unifying column names

We keep only the columns relevant for our computation, these are the Date because we want to compute a monthly average of daily new cases and deaths, so we must be able to differentiate different days and months. We keep the Variable because we will need it to identify the rows relevant to new cases and new deaths. We keep the National columns because we are interested by values per country so we don't need to know the details per region specifically but just nation wide.

In [ ]:
frameGU = frameGU[['Date', 'Variable', 'National']] #we keep only the relevant columns needed for our final computation
frameSL = frameSL[['Date', 'Variable', 'National']] 
frameLI = frameLI[['Date', 'Variable', 'National']] 

Let us now observe the Variable values to decide which one are relevant for our computation.

In [ ]:
frameSL.Variable.unique()

In [ ]:
frameLI.Variable.unique()

In [ ]:
frameGU.Variable.unique()

We notice that the Variables are very different in each data frame. Since we are interested by new cases and new deaths per day, we choose to keep only the confirmed new cases/deaths so as not to be biased by bad data. These variables are written in a more compact form in the Sierra Leone frame so we choose to keep their formulation for the other dataframes. For Sierra Leone the relevant variables are 'new_confirmed' and 'death_confirmed'. For Liberia it is 'Total death/s in confirmed cases' and 'New case/s (confirmed)'. For Guinea it is 'New cases of confirmed' and 'New deaths registered today (confirmed)'. However, we notice that the values for confirmed deaths in Liberia and Sierra Leone are cumulative so for a particular date we must subtract the value of the previous date so that we have only the new deaths.

For Sierra Leone, we start by keeping only the rows corresponding to the confirmed deaths and we remove the rows with missing values. We change the type of values in the column 'National' from string to integer and subtract to each value the previous one so that the resulting column does not have cumulative confirmed deaths but daily confirmed deaths. Once again, we remove the rows with missing data (the first row won't have any relevant data as we have no reference to subtract to it). Finally we create a single frame for Sierra Leone with our computed confirmed daily deaths and new confirmed daily cases that is sorted and indexed by the Date column.

In [ ]:
frameSLDeaths = frameSL.loc[frameSL['Variable'].isin(['death_confirmed'])]
frameSLDeathsNA = frameSLDeaths.dropna() #we remove rows with missing values
frameSLDeathsNA.National = frameSLDeathsNA.National.apply(pd.to_numeric) #cast National values to integers
frameSLDeathsNA.National=frameSLDeathsNA.National-frameSLDeathsNA.National.shift(1) #non cumulative deaths
frameSLDeathsNA = frameSLDeathsNA.dropna() #we remove the first row with missing value
frameSLNewNA = frameSL.loc[frameSL['Variable'].isin(['new_confirmed'])].dropna()
frameTmp = [frameSLDeathsNA, frameSLNewNA]
frameSL = pd.concat(frameTmp)
frameSL.index=frameSL.Date
frameSL = frameSL.sort_index()

We do the exact same thing for Liberia: Compute the non cumulative deaths and create a single dataframe with the daily confirmed deaths and new cases that is sorted and indexed by dates.

In [ ]:
frameLIDeaths = frameLI.loc[frameLI['Variable'].isin(['Total death/s in confirmed cases'])]
frameLIDeathsNA = frameLIDeaths.dropna() #we remove rows with missing values
frameLIDeathsNA.National = frameLIDeathsNA.National.apply(pd.to_numeric) #cast National values to integers
frameLIDeathsNA.National=frameLIDeathsNA.National-frameLIDeathsNA.National.shift(1) #non cumulative deaths
frameLIDeathsNA = frameLIDeathsNA.dropna() #we remove the first row with missing value
frameLINewNA = frameLI.loc[frameLI['Variable'].isin(['New case/s (confirmed)'])]
frameLINewNA=frameLINewNA.dropna()
frameTmp = [frameLIDeathsNA, frameLINewNA]
frameLI = pd.concat(frameTmp)
frameLI.index=frameLI.Date
frameLI = frameLI.sort_index()

The death values for Guinea are already non cumulative, so we just create a single dataframe with the new deaths and cases confirmed like for the two previous countries.

In [ ]:
frameGU = frameGU.loc[frameGU['Variable'].isin( ['New cases of confirmed', 'New deaths registered today (confirmed)'])]
frameGU = frameGU.dropna() #we remove the missing values
frameGU.index=frameGU.Date
frameGU = frameGU.sort_index()

We now create a single dataframe using the dataframes per country created above. We concatenate the previous frames and make them accessible through keys that correspond to the right country. We rename the Variable values so that they are consistent throughout countries: all the new daily confirmed cases and deaths are described by 'new_confirmed' and 'deaths_confirmed' respectively. 

In [ ]:
frameTmp = [frameGU, frameSL, frameLI]
frame = pd.concat(frameTmp, keys=['Guinea', 'Sierra Leone', 'Liberia'])

print(frame.Variable.unique())

varMapping = {
    'New cases of confirmed':                 'new_confirmed',
    'New deaths registered today (confirmed)':'death_confirmed',
    'New case/s (confirmed)':                 'new_confirmed',
    'Total death/s in confirmed cases':       'death_confirmed'
}

frame.Variable = frame.Variable.apply(lambda v: varMapping.get(v, v))
    
print(frame.Variable.unique())

In the display of the resulting frame, we notice outlier values. Indeed some are negative and some are unreasonably high. We decide to remove the rows corresponding to such values. It does not make sense to set them to zero because our computation will be a daily average per month computed as a mean of the available values. We cast all the values in the National column to integer so that we can compare the values to the if conditions in order to keep only does within a reasonable boundary.

In [ ]:
frame['National'] = frame['National'].astype(int)
frame = frame[["Variable", "National"]].reset_index()
frame = frame[(frame['National'] >=0) & (frame['National'] < 200)]
frame

All the values now seem reasonable to use to perform our computation. We reformat the dates so that we can use them to group the deaths and new cases values per country per month and compute a mean over the days.

In [ ]:
frame['DateMonth'] = frame['Date'].apply(lambda x: str(x.year) + "-" + str(x.month))
deaths = frame[frame['Variable'] == "death_confirmed"].groupby(["level_0", "DateMonth"]).mean()["National"]
new_cases = frame[frame['Variable'] == "new_confirmed"].groupby(["level_0", "DateMonth"]).mean()["National"]

We now display the resulting frame with the daily average per month of new deaths per country. 

In [ ]:
deaths

We now display the resulting frame with the daily average per month of new cases per country.

In [ ]:
new_cases

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

ANSWER:
We first import all the spreadsheets in separate dataframes.

In [ ]:
import pandas as pd

mbs = []
for i in range(1,10):
    temp = pd.read_excel(DATA_FOLDER+'/microbiome/MID'+str(i)+'.xls', sheetname='Sheet 1', header=None)
    temp.columns = ['name', 'MID'+str(i)] # Using file name as column name for easier merging
    mbs.append(temp)
   
    
metadata = pd.read_excel(DATA_FOLDER+'/microbiome/metadata.xls', sheetname='Sheet1')

we verify that the name column is a unique index for all the MID data and set this column as the index

In [ ]:
mbs_ind = []
doublons = False
for i in range(1,10):
    mbs_ind.append(mbs[i-1].set_index(['name']))
    if mbs_ind[i-1].index.is_unique:
        mbs[i-1] = mbs_ind[i-1]
    else:
        print("Table " + i + " contains doublons")
        doublons = True

if not doublons:
    print("Tables are indexed by the column name")

So we can merge the MID tables using pandas' merge fucntion

In [ ]:
import functools as ft
mbs_merged = ft.reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), mbs)
mbs_merged.head()

now, we use the metadata file to have meaningfull headers for our data

In [ ]:
metadata

we start be reordering the columns according to the metadata

In [ ]:
mbs_order = mbs_merged[metadata.BARCODE]
mbs_order.head()

we split the table in two because the first column is 'EXTRACTION CONTROL' and does not have different types (tissue or stools)

In [ ]:
mbs_part1 = mbs_order.iloc[:, :1]
mbs_part2 = mbs_order.iloc[:, 1:]
mbs_part2.head()

we create the 2-levels index according to the metadata.

In [ ]:
header = pd.MultiIndex.from_product([['NEC1','control1','NEC2','control2'],
                                     ['tissue','stool']],
                                    names=['Test','Type'])
mbs_part2.columns = header

mbs_part1.columns = pd.MultiIndex.from_product([['EXTRACTION CONTROL'],['']])

merge again all the columns together

In [ ]:
mbs_nans = pd.merge(mbs_part1,mbs_part2,left_index=True, right_index=True, how='outer')
mbs_nans.head()

We first check the required invariants on index and dtypes, because it is easier before replacing NaNs by "unknown". Then we perform the replacement.

In [ ]:
from numpy import issubdtype, number
print('Index are unique:', mbs_nans.index.is_unique)
print('All entries are either a number or nan:', issubdtype(mbs_nans.dtypes.all(), number))
mbs_final = mbs_nans.fillna("unknown")

We print the result:

In [ ]:
mbs_final

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
data = pd.read_excel(DATA_FOLDER+'/titanic.xls', sheetname='titanic')

In [ ]:
data.pclass.unique()

In [ ]:
data.survived.unique()

In [ ]:
data.sex.unique()

In [ ]:
data.embarked.unique()

In [ ]:
data.boat.unique()

In [ ]:
data.cabin.isnull().sum()

In [ ]:
len(data[data.body.notnull()].body.unique()) == len(data[data.body.notnull()].body)

In [ ]:
categories = {
    'pclass': 'category',
    #'survived': 'category',
    'sex': 'category',
    'embarked': 'category',
}

cities = {
    "C":"Cherbourg",
    "Q":"Queenstown",
    "S":"Southampton"
}

passengers = pd.read_excel(DATA_FOLDER+'/titanic.xls', sheetname='titanic', dtype=categories)

passengers.embarked.cat.categories = [cities[c] for c in passengers.embarked.cat.categories]
passengers.embarked.cat.add_categories(["Unknown"], inplace=True)
passengers.embarked.fillna("Unknown", inplace=True)

passengers.survived = passengers.survived.astype('category', categories=[0, 1], ordered=True)

# Should inlcude unknown ages ? 
passengers["age_cat"] = pd.cut(passengers.age, range(0, np.ceil(passengers.age.max()).astype('int')+1, 10))

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(15,15))
sns.countplot(x="pclass", data=passengers, ax=ax1);
sns.countplot(x='embarked', data=passengers, ax=ax2);
sns.countplot(x='sex', data=passengers, ax=ax3);
sns.countplot(x='age_cat', data=passengers, ax=ax4);

In [ ]:
def get_floor(o):
    if type(o) != str:
        return np.NaN
    else:
        floor_codes = list(set([c for c in o if c.isalpha()]))
        return floor_codes[0] if len(floor_codes) == 1 else np.NaN

passengers['floor'] = passengers.cabin.transform(get_floor).astype('category')

fig, axs = plt.subplots(figsize=(5,5))
passengers.floor.value_counts(sort=False).plot.pie(); #  We could have better precision by infering  missing value from class

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(15,5))
colors = ['lightcoral','darkseagreen']
labels = ['Died', 'Survived']
for (g, s), ax in zip(passengers.groupby(['pclass']).survived, axs):
    s.value_counts(sort=False).plot.pie(ax=ax, colors=colors, labels=labels)

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
width = 0.25
for i, (grp_lab, grp) in enumerate(passengers.groupby(["pclass", "sex"]).survived):
    shift = (grp_lab[1]=='female')*(width+0.01)
    down = plt.bar(grp_lab[0]+shift, grp.value_counts()[0], width, color=colors[0])
    up = plt.bar(grp_lab[0]+shift, grp.value_counts()[1], width, bottom=grp.value_counts()[0], color=colors[1])

ax.set_xticks(passengers.pclass.cat.categories+0.5*width)
ax.set_xticklabels(passengers.pclass.cat.categories)

ax.set_title('Surviva count per class (Male, Female)')
ax.set_ylabel('Count')
ax.set_xlabel('Class (Male, Female)')
plt.legend(['Died', 'Survived'], loc='upper left');

Create 2 equally populated age categories and calculate survival proportions by age category, travel class and sex. Present your results in a DataFrame with unique index.

In [ ]:
print(passengers.age.isnull().sum())
print(passengers.age.mean())
pd.qcut(passengers.age, [0, 0.5, 1]).value_counts()